In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from itertools import combinations
import pulp
import ast

In [ ]:
#Definición de parámetros de negocio de operaciones
lista_estaciones= pd.DataFrame({'estaciones': ["NA16", "NA01", "NA02", "NA06","NA07","NA14","NA15","NA17","NA18","PNOR","NARL","CHOC","RCAS"]})
Tlim_max_tramo = timedelta(hours=6, minutes=30)
Tlim_min_tramo = timedelta(hours=1, minutes=20)
today = pd.Timestamp.today().normalize()  # Obtener fecha de hoy sin hora
Tlim_alm_min = today + timedelta(hours=12, minutes=20)
Tlim_alm_max = today + timedelta(hours=14, minutes=40)
Tlim_ro_min= today + timedelta(hours=8, minutes=00)
Tlim_ro_max= today + timedelta(hours=22, minutes=00)

In [ ]:
#Lectura de documento + limpieza de horas
def lectura_csv(path_file):
    df = pd.read_csv('TRO_HABIL_S49.csv', parse_dates=['hora_inicio', 'hora_fin'], sep=";")
    limit=timedelta(hours=3)
    date_limit=today+limit
    for j in df.index:
        if df.loc[j,"hora_inicio"]<=date_limit:
            df.loc[j,"hora_inicio"]=df.loc[j,"hora_inicio"]+timedelta(days=1)
        else:
            df.loc[j,"hora_inicio"]=df.loc[j,"hora_inicio"]
        if df.loc[j,"hora_fin"]<=date_limit:
            df.loc[j,"hora_fin"]=df.loc[j,"hora_fin"]+timedelta(days=1)
        else:
            df.loc[j,"hora_fin"]=df.loc[j,"hora_fin"]

In [ ]:
#Función de filtro de los relief opportunity (RO) por CC (bus ficiticio)
def filtro_ro(df,lista_estaciones):
    lista_ro = df[df['estación_fin'].isin(lista_estaciones['estaciones'])].sort_values(by='hora_inicio')
    df_sal = df.loc[df["hora_inicio"].idxmin()]
    df_enc = df.loc[df["hora_fin"].idxmax()]
    return lista_ro,df_sal,df_enc

In [ ]:
#Confeccion de la base de Relief opportunity (RO)
def frame_ro(df,lista_estaciones):
    #Extracción del primer punto con PNOR
    lista_perm_PNOR = df[df['estacion_inicio'] == "PNOR"][['id_viaje','CC', 'hora_inicio', 'estacion_inicio']].copy()
    lista_perm_PNOR.rename(columns={'hora_inicio': 'hora_ro', 'estacion_inicio': 'estacion_ro'}, inplace=True)
    
    #Extracción de las estaciones de destino presentes en la lista 'estaciones'
    lista_perm_rest = df[df['estación_fin'].isin(lista_estaciones['estaciones'])& (df['estacion_inicio']!="PNOR")][['id_viaje','CC', 'hora_fin', 'estación_fin']].copy()
    lista_perm_rest.rename(columns={'hora_fin': 'hora_ro', 'estación_fin': 'estacion_ro'}, inplace=True)

    #Extracción del último punto con PNOR
    lista_PNOR_fin = df[df['estación_fin']=="PNOR"][['id_viaje','CC', 'hora_fin', 'estación_fin']].copy()
    lista_PNOR_fin.rename(columns={'hora_fin': 'hora_ro', 'estación_fin': 'estacion_ro'}, inplace=True)

    #Concatenar 3 listas y reemplazo de nombres de estación Naranjal (alimentador)
    lista_ro = pd.concat([lista_perm_PNOR, lista_perm_rest,lista_PNOR_fin], ignore_index=True)
    lista_ro_fin=lista_ro.sort_values(by=['CC','hora_ro']).drop_duplicates().reset_index(drop=True)
    lista_ro_fin.replace({'estacion_ro':{'NA16':'NARL','NA02':'NARL','NA02':'NARL', 'NA06':'NARL','NA07':'NARL','NA14':'NARL','NA15':'NARL','NA17':'NARL','NA18':'NARL'}},inplace=True)

    return lista_ro_fin



In [6]:
#Selección de RO's pertenecientes a CC con tiempos mayores a T_lim_max_tramo
def sel_ro_excl_Tmax(df):
    CC_excl_RO=df.groupby("CC").filter(lambda x: x["hora_ro"].max() - x["hora_ro"].min() < Tlim_max_tramo)["CC"].unique().astype(int)
    lista_ro_excl=df.groupby("CC").filter(lambda x: x["hora_ro"].max() - x["hora_ro"].min() < Tlim_max_tramo)
    df_lista_ro_excl=pd.DataFrame()
    for p in CC_excl_RO:
        lista_ro_excl_min=lista_ro_excl[lista_ro_excl["CC"]==p].loc[[lista_ro_excl[lista_ro_excl["CC"]==p]["hora_ro"].idxmin()]]
        lista_ro_excl_max=lista_ro_excl[lista_ro_excl["CC"]==p].loc[[lista_ro_excl[lista_ro_excl["CC"]==p]["hora_ro"].idxmax()]]
        df_lista_ro_excl=pd.concat([df_lista_ro_excl,lista_ro_excl_min,lista_ro_excl_max],ignore_index=True)
    return df_lista_ro_excl

In [ ]:
#Descarte de RO que estan fuera de rango de un corte mínimo o máximo
def sel_ro_excl_Tlim(df):
    df_ro_excl_Tlim=df[(df['hora_ro']>Tlim_ro_min) & (df['hora_ro']<Tlim_ro_max)]

    return df_ro_excl_Tlim


## Formación iterativa de tramos de trabajo

In [ ]:
#Iteración de tramos de trabajo
def form_tramos(df):
    # Lista para guardar las combinaciones válidas
    combinaciones_validas = []

    # Recorrer cada grupo por CC
    for cc, grupo in df.groupby("CC"):
        grupo = grupo.sort_values("hora_ro").reset_index(drop=True)

        for i, j in combinations(grupo.index, 2):
            t1 = grupo.loc[i, "hora_ro"]
            t2 = grupo.loc[j, "hora_ro"]
            diff = t2 - t1

            if pd.Timedelta(hours=1,minutes=20) <= diff <= pd.Timedelta(hours=6, minutes=30):
                combinaciones_validas.append({
                    "CC": cc,
                    "hora_inicio": t1,
                    "estacion_inicio": grupo.loc[i, "estacion_ro"],
                    "hora_fin": t2,
                    "estacion_fin": grupo.loc[j, "estacion_ro"],
                    "duracion": diff,
                    "id_viaje_inicio": grupo.loc[i, "id_viaje"],
                    "id_viaje_fin": grupo.loc[j, "id_viaje"]
                })
    df_tramos_tr=pd.DataFrame(combinaciones_validas)
    return df_tramos_tr

In [ ]:
#Agregar la columna a los tramos de trabajo en el que figura la lista de viajes que acapara el tramo
def generar_lista_ids(row):
    inicio = int(row['id_viaje_inicio'])
    fin = int(row['id_viaje_fin'])

    # Rango natural (sin incluir inicio)
    lista = list(range(inicio + 1, fin + 1))

    # Si inicio es múltiplo de 100 + 1 → incluir inicio
    if (inicio - 1) % 100 == 0:
        lista.insert(0, inicio)

    return lista

In [ ]:
#Eliminar los tramos que no cumplan con las restricciones de almuerzo
def restr_almuerzo(df):
    df_restr_alm=df[(df["hora_inicio"]<Tlim_alm_min) & (df["hora_fin"]>Tlim_alm_max)]
    df_restr_no_alm=df[~df.index.isin(df_restr_alm.index)]
    return df_restr_no_alm

## Formación iterativa de jornadas de trabajo

In [1]:
#Formación de Jornadas turno regular, misma estación
def jornadas_regular_eq_est (df):
    # Ordenamos por hora de inicio
    df_combinaciones_regular = df.sort_values("hora_inicio").reset_index(drop=True)

    combinaciones_encadenadas = []

# Recorremos todas las combinaciones de pares (i, j) donde j > i
    for i in range(len(df_combinaciones_regular) - 1):
        fila_i = df_combinaciones_regular.loc[i]

        for j in range(i + 1, len(df_combinaciones_regular)):
            fila_j = df_combinaciones_regular.loc[j]

            # Estación debe coincidir
            misma_estacion = fila_i["estacion_fin"] == fila_j["estacion_inicio"]

            # Tiempo entre fin del tramo 1 e inicio del tramo 2
            diff_espera = fila_j["hora_inicio"] - fila_i["hora_fin"]
            espera_valida = timedelta(minutes=49) <= diff_espera <= timedelta(hours=1, minutes=30)

            # Suma de duración total
            duracion_total = fila_i["duracion"] + fila_j["duracion"]
            duracion_valida = timedelta(hours=6,minutes=30) <= duracion_total <= timedelta(hours=9, minutes=15)

            if misma_estacion and espera_valida and duracion_valida:
                combinaciones_encadenadas.append({
                    "CC_1": fila_i["CC"],
                    "hora_inicio_1": fila_i["hora_inicio"],
                    "estacion_inicio_1": fila_i["estacion_inicio"],
                    "hora_fin_1": fila_i["hora_fin"],
                    "estacion_fin_1": fila_i["estacion_fin"],
                    "CC_2": fila_j["CC"],
                    "hora_inicio_2": fila_j["hora_inicio"],
                    "estacion_inicio_2": fila_j["estacion_inicio"],
                    "hora_fin_2": fila_j["hora_fin"],
                    "estacion_fin_2": fila_j["estacion_fin"],
                    "espera_entre_tramos": diff_espera,
                    "duracion_total": duracion_total,
                    "lista_id_viaje":fila_i["lista_id_viaje"]+fila_j["lista_id_viaje"]
                })

    # Creamos el DataFrame con los resultados
    df_jorn_eq_regular = pd.DataFrame(combinaciones_encadenadas)
    df_jorn_eq_regular["tipo_turno"]="Regular"
    return df_jorn_eq_regular

In [2]:
#Formación de Jornadas turno partido
max_hora_partido=timedelta(hours=22,minutes=30)

def jornadas_partido_eq_est (df):
    df_combinaciones = df[df["hora_fin"]<max_hora_partido]
    df_combinaciones_partido = df_combinaciones.sort_values("hora_inicio").reset_index(drop=True)

    resultados_filtro_avanzado = []

    for idx_a in range(len(df_combinaciones_partido) - 1):
        tramo_a = df_combinaciones_partido.loc[idx_a]

        for idx_b in range(idx_a + 1, len(df_combinaciones_partido)):
            tramo_b = df_combinaciones_partido.loc[idx_b]

            # Verificamos duración total combinada
            tiempo_total = tramo_a["duracion"] + tramo_b["duracion"]
            if not (timedelta(hours=7,minutes=30) <= tiempo_total <= timedelta(hours=9, minutes=49)):
                continue

            # Calculamos la espera entre tramos
            lapso_descanso = tramo_b["hora_inicio"] - tramo_a["hora_fin"]

            misma_base = tramo_a["estacion_fin"] == tramo_b["estacion_inicio"]
            if misma_base:
                if lapso_descanso <= timedelta(hours=1, minutes=42):
                    continue
            else:
                if lapso_descanso <= timedelta(hours=2, minutes=40):
                    continue

            hora_limite_part = datetime.combine(tramo_a["hora_inicio"].date(), datetime.strptime("22:30", "%H:%M").time())
            if tramo_b["hora_fin"] > hora_limite_part:
                continue

            # Validamos límite horario del segundo tramo
            '''
            if tramo_b["hora_fin"].time() > pd.to_datetime("22:30").time():
                continue
            '''
            # Si pasa todos los filtros, lo agregamos
            resultados_filtro_avanzado.append({
                "CC_1": tramo_a["CC"],
                "hora_inicio_1": tramo_a["hora_inicio"],
                "estacion_inicio_1": tramo_a["estacion_inicio"],
                "hora_fin_1": tramo_a["hora_fin"],
                "estacion_fin_1": tramo_a["estacion_fin"],

                "CC_2": tramo_b["CC"],
                "hora_inicio_2": tramo_b["hora_inicio"],
                "estacion_inicio_2": tramo_b["estacion_inicio"],
                "hora_fin_2": tramo_b["hora_fin"],
                "estacion_fin_2": tramo_b["estacion_fin"],

                "espera_entre_tramos": lapso_descanso,
                "duracion_total": tiempo_total,
                "lista_id_viaje":tramo_a["lista_id_viaje"]+tramo_b["lista_id_viaje"]
            })

    # Convertimos a DataFrame
    df_avanzado = pd.DataFrame(resultados_filtro_avanzado)
    df_avanzado["tipo_turno"]="Partido"
    return df_avanzado

NameError: name 'timedelta' is not defined

In [ ]:
#Formación de empalmes en estaciones diferentes

traslados_validos = {
    ("NARL", "RCAS"): {
        "tiempo": timedelta(minutes=25),
        "modos": {"adelantar_2"}   # 👈 SOLO adelantar tramo 2
    },
    ("RCAS", "NARL"): {
        "tiempo": timedelta(minutes=25),
        "modos": {"extender_1"}    # 👈 SOLO extender tramo 1
    },
}

def jornadas_regular_diff_est (df_combinaciones,traslados_validos,ESPERA_MIN,ESPERA_MAX,DUR_MIN,DUR_MAX):
    df = df_combinaciones.sort_values("hora_inicio").reset_index(drop=True)

    df["lista_id_viaje"] = df["lista_id_viaje"].apply(
        lambda x: x if isinstance(x, list) else []
    )

    resultados = []

    for i in range(len(df) - 1):
        t1 = df.loc[i]

        for j in range(i + 1, len(df)):
            t2 = df.loc[j]

            if t1["CC"] == t2["CC"]:
                continue

            est_fin_1 = t1["estacion_fin"]
            est_ini_2 = t2["estacion_inicio"]

            if est_fin_1 == est_ini_2:
                continue

            clave = (est_fin_1, est_ini_2)
            if clave not in traslados_validos:
                continue

            traslado_info = traslados_validos[clave]
            traslado = traslado_info["tiempo"]
            modos_validos = traslado_info["modos"]

            # ==================================================
            # ESCENARIO A → ADELANTAR INICIO TRAMO 2
            # ==================================================
            if "adelantar_2" in modos_validos:

                nuevo_inicio_2 = t2["hora_inicio"] - traslado

                if nuevo_inicio_2 >= t1["hora_fin"]:

                    espera_total = nuevo_inicio_2 - t1["hora_fin"]

                    duracion_total = (
                        (t1["hora_fin"] - t1["hora_inicio"]) +
                        (t2["hora_fin"] - nuevo_inicio_2)
                    )

                    if ESPERA_MIN <= espera_total <= ESPERA_MAX and \
                    DUR_MIN <= duracion_total <= DUR_MAX:

                        resultados.append({
                            "CC_1": t1["CC"],
                            "hora_inicio_1": t1["hora_inicio"],
                            "estacion_inicio_1": t1["estacion_inicio"],
                            "hora_fin_1": t1["hora_fin"],
                            "estacion_fin_1": est_fin_1,

                            "CC_2": t2["CC"],
                            "hora_inicio_2": nuevo_inicio_2,
                            "estacion_inicio_2": est_fin_1,
                            "hora_fin_2": t2["hora_fin"],
                            "estacion_fin_2": t2["estacion_fin"],

                            "tipo_ajuste": "adelantar_2",
                            "traslado": traslado,
                            "espera_total": espera_total,
                            "duracion_total": duracion_total,
                            "lista_id_viaje": t1["lista_id_viaje"] + t2["lista_id_viaje"]
                        })

            # ==================================================
            # ESCENARIO B → EXTENDER FIN TRAMO 1
            # ==================================================
            if "extender_1" in modos_validos:

                nuevo_fin_1 = t1["hora_fin"] + traslado

                if t2["hora_inicio"] >= nuevo_fin_1:

                    espera_total = t2["hora_inicio"] - nuevo_fin_1

                    duracion_total = (
                        (nuevo_fin_1 - t1["hora_inicio"]) +
                        (t2["hora_fin"] - t2["hora_inicio"])
                    )

                    if ESPERA_MIN <= espera_total <= ESPERA_MAX and \
                    DUR_MIN <= duracion_total <= DUR_MAX:

                        resultados.append({
                            "CC_1": t1["CC"],
                            "hora_inicio_1": t1["hora_inicio"],
                            "estacion_inicio_1": t1["estacion_inicio"],
                            "hora_fin_1": nuevo_fin_1,
                            "estacion_fin_1": est_ini_2,

                            "CC_2": t2["CC"],
                            "hora_inicio_2": t2["hora_inicio"],
                            "estacion_inicio_2": est_ini_2,
                            "hora_fin_2": t2["hora_fin"],
                            "estacion_fin_2": t2["estacion_fin"],

                            "tipo_ajuste": "extender_1",
                            "traslado": traslado,
                            "espera_entre_tramos": espera_total,
                            "duracion_total": duracion_total,
                            "lista_id_viaje": t1["lista_id_viaje"] + t2["lista_id_viaje"]
                        })

    df_empalmes_diff = pd.DataFrame(resultados)

    return df_empalmes_diff

## Selección de jornadas de trabajo que cumplan objetivos

In [ ]:
def set_covering4schedule(df_id_viajes,df_jornadas):
    # Si lista_id_viaje viene como string, convertirlo a lista real
    df_jornadas['lista_id_viaje'] = df_jornadas['lista_id_viaje'].apply(
        lambda x: ast.literal_eval(str(x)) if isinstance(x, str) else x
    )

    # Universo de viajes a cubrir
    U = set(df_id_viajes['id_viaje'].unique())

    print(f"Total de viajes a cubrir: {len(U)}")
    print(f"Total de jornadas candidateadas: {len(df_jornadas)}")


    # ===========================================================
    # 2. DEFINICIÓN DEL MODELO SET COVERING
    # ===========================================================

    model = pulp.LpProblem("SetCovering_Jornadas", pulp.LpMinimize)

    # Índices de jornadas
    J = df_jornadas.index.tolist()

    # Variable binaria: x_j = 1 si seleccionamos la jornada j
    x = pulp.LpVariable.dicts('x', J, lowBound=0, upBound=1, cat='Binary')

    # Objetivo: minimizar número total de jornadas seleccionadas
    model += pulp.lpSum([x[j] for j in J])


    # ===========================================================
    # 3. RESTRICCIONES: CADA VIAJE DEBE SER CUBIERTO UNA VEZ
    # ===========================================================

    for v in U:
        jornadas_que_cubren = [j for j in J if v in df_jornadas.loc[j, 'lista_id_viaje']]

        if len(jornadas_que_cubren) == 0:
            print(f"⚠ Advertencia: el id_viaje {v} no está cubierto por ninguna jornada")

        model += pulp.lpSum([x[j] for j in jornadas_que_cubren]) == 1


    # ===========================================================
    # 4. RESOLVER
    # ===========================================================

    print("\n⏳ Resolviendo el SCP...")
    solver = pulp.PULP_CBC_CMD(msg=True)
    model.solve(solver)

    print(f"\nEstado del modelo: {pulp.LpStatus[model.status]}")


    # ===========================================================
    # 5. RESULTADOS
    # ===========================================================

    if pulp.LpStatus[model.status] == 'Optimal':

        seleccionadas = [j for j in J if x[j].value() == 1]
        print(f"\nJornadas seleccionadas: {len(seleccionadas)}")

        df_sol = df_jornadas.loc[seleccionadas].copy()
        df_sol.reset_index(drop=True, inplace=True)

        print(df_sol.head())

        # Guardar solución
        df_sol.to_csv("solucion_jornadas_scp.csv", index=False)
        print("\n✅ Solución guardada en solucion_jornadas_scp.csv")

    else:
        print("\n❌ El modelo NO encontró solución óptima.")
        print("Puedes extraer la información de qué viajes quedaron sin cubrir:")

        # Chequear viajes sin cubrir (en caso de modelo no óptimo o infactible)
        viajes_sin_cubrir = []
        for v in U:
            if sum(x[j].value() for j in J if v in df_jornadas.loc[j, 'lista_id_viaje']) == 0:
                viajes_sin_cubrir.append(v)

        print("Viajes sin cubrir:", viajes_sin_cubrir)